Quick Note: Items Scraped:

- Job Title
- Company Name
- Company Location
- Job Description
- Work Method (Hybrid, Remote, On-Site)
- Experience Level
- Post Date

In [1]:
'''
    "Scraping Everything from Linkedin" # 1

    Author: aiXpertLab@gmail.com
    Date: January 14, 2024
    Description: 
    This is a Python script for scraping job lists including details from Linkedin.
    It's one of "Scraping Everything from Linkedin"

'''

# Libraries
import os
import pandas as pd    

from dotenv import load_dotenv
from selenium import webdriver
from selenium.webdriver.common.by import By

load_dotenv()
user_name = os.getenv('LD_USR')
password  = os.getenv('LD_KEY')

driver = webdriver.Chrome()  
driver.implicitly_wait(511)
driver.get('https://www.linkedin.com/login')

driver.find_element(By.XPATH, '//*[@id="username"]').send_keys(user_name)
driver.find_element(By.XPATH, '//*[@id="password"]').send_keys(password)
driver.find_element(By.XPATH, "//button[@type='submit']").click()   # Login button
# driver.find_element(By.XPATH, '//*[@id="global-nav"]/div/nav/ul/li[3]/a/span').click()   #jobs

# driver.get("https://www.linkedin.com/jobs/search/?currentJobId=3800581161&geoId=105080838&keywords=Artificial%20Intelligence%20Engineer&location=New%20York%2C%20United%20States&origin=JOB_SEARCH_PAGE_SEARCH_BUTTON&refresh=true&sortBy=R")

# above, Login to the Linkedin, and get the search results.
# Get all links for these offers
links = []
# Navigate 13 pages
print('Links are being collected now.')
try: 
    jobs_block = driver.find_element(By.CLASS_NAME, 'jobs-search-results-list')
    jobs_list= jobs_block.find_elements(By.CSS_SELECTOR, '.jobs-search-results__list-item')

    for job in jobs_list:
        all_links = job.find_elements(By.TAG_NAME,'a')
        for a in all_links:
            if str(a.get_attribute('href')).startswith("https://www.linkedin.com/jobs/view") and a.get_attribute('href') not in links: 
                links.append(a.get_attribute('href'))
            else:
                pass
        # scroll down for each job element
        driver.execute_script("arguments[0].scrollIntoView();", job)

except:
    pass
print('Found ' + str(len(links)) + ' links for job offers')
# Create empty lists to store information
job_titles = []
company_names = []
company_locations = []
work_methods = []
post_dates = []
work_times = [] 
job_desc = []

# i = 0
j = 1


Links are being collected now.
Found 0 links for job offers


In [2]:
# Visit each link one by one to scrape the information
print('Visiting the links and collecting information just started.')
range_number = min(5, len(links))
for i in range(range_number):
    driver.get(links[i])
    # i=i+1
    # Click See more.
    # driver.find_element(By.CLASS_NAME,"artdeco-button__text").click()
    # see_more_button = driver.find_element(By.CSS_SELECTOR, 'button[aria-label="Click to see more description"].jobs-description__footer-button')
    # see_more_button  = driver.find_element(By.CSS_SELECTOR, 'button[id="ember37"].jobs-description__footer-button')
    try:
        see_more_button  = driver.find_element(By.ID,'ember37')
        see_more_button.click()
    except:
         pass

    # Find the general information of the job offers
    contents = driver.find_elements(By.CLASS_NAME,'p5')
    print(contents)
    for content in contents:
     #   try:
            job_title_element = driver.find_element(By.CSS_SELECTOR, 'h1.job-details-jobs-unified-top-card__job-title')
            company_name_element = driver.find_element(By.CSS_SELECTOR, 'div.job-details-jobs-unified-top-card__primary-description-container a.app-aware-link')
            company_location_element = driver.find_element(By.CSS_SELECTOR, 'div.job-details-jobs-unified-top-card__primary-description-container span.tvm__text--neutral')

            job_titles.append(job_title_element.text.strip())
            company_names.append(company_name_element.text.strip())
            company_locations.append(company_location_element.text.strip())

            print(job_titles)
            print(company_names)
            print(f'Scraping the Job Offer {j} DONE.')
            j+= 1
      #  except:
       #     pass
        
        # Scraping the job description
    job_description = driver.find_elements(By.CLASS_NAME, 'jobs-description__content')
    for description in job_description:
        job_text = description.find_element(By.CLASS_NAME, "jobs-box__html-content").text
        job_desc.append(job_text)
        print(job_desc)
        print(f'Scraping the Job Offer {j}')
            
# Creating the dataframe 
df = pd.DataFrame(list(zip(job_titles,company_names,
                    company_locations,work_methods,
                    post_dates,work_times)),
                    columns =['job_title', 'company_name',
                           'company_location','work_method',
                           'post_date','work_time'])

# Storing the data to csv file
df.to_csv('job_offers.csv', index=False)

# Output job descriptions to txt file
with open('job_descriptions.txt', 'w',encoding="utf-8") as f:
    for line in job_desc:
        f.write(line)
        f.write('\n')

Visiting the links and collecting information just started.
